# 第9章: ベクトル空間法 (I)

[enwiki-20150112-400-r10-105752.txt.bz2](data/enwiki-20150112-400-r10-105752.txt.bz2)は，2015年1月12日時点の英語のWikipedia記事のうち，約400語以上で構成される記事の中から，ランダムに1/10サンプリングした105,752記事のテキストをbzip2形式で圧縮したものである．このテキストをコーパスとして，単語の意味を表すベクトル（分散表現）を学習したい．第9章の前半では，コーパスから作成した単語文脈共起行列に主成分分析を適用し，単語ベクトルを学習する過程を，いくつかの処理に分けて実装する．第9章の後半では，学習で得られた単語ベクトル（300次元）を用い，単語の類似度計算やアナロジー（類推）を行う．

なお，問題83を素直に実装すると，大量（約7GB）の主記憶が必要になる．
メモリが不足する場合は，処理を工夫するか，1/100サンプリングのコーパス[enwiki-20150112-400-r100-10576.txt.bz2](data/enwiki-20150112-400-r100-10576.txt.bz2)を用いよ．


In [13]:
#!wget -nc http://www.cl.ecei.tohoku.ac.jp/nlp100/data/enwiki-20150112-400-r10-105752.txt.bz2 -P ./data/
#!wget -nc http://www.cl.ecei.tohoku.ac.jp/nlp100/data/enwiki-20150112-400-r100-10576.txt.bz2 -P ./data/

In [14]:
#!bzip2 -d ./data/enwiki-20150112-400-r10-105752.txt.bz2
#!bzip2 -d ./data/enwiki-20150112-400-r100-10576.txt.bz2

In [58]:
!head -3 ./data/enwiki-20150112-400-r100-10576.txt

Anarchism

Anarchism is a political philosophy that advocates stateless societies often defined as self-governed voluntary institutions, but that several authors have defined as more specific institutions based on non-hierarchical free associations. Anarchism holds the state to be undesirable, unnecessary, or harmful. While anti-statism is central, anarchism entails opposing authority or hierarchical organisation in the conduct of human relations, including, but not limited to, the state system.


## 80. コーパスの整形

文を単語列に変換する最も単純な方法は，空白文字で単語に区切ることである．
ただ，この方法では文末のピリオドや括弧などの記号が単語に含まれてしまう．
そこで，コーパスの各行のテキストを空白文字でトークンのリストに分割した後，各トークンに以下の処理を施し，単語から記号を除去せよ．

+ トークンの先頭と末尾に出現する次の文字を削除: `.,!?;:()[]'"`
+ 空文字列となったトークンは削除

以上の処理を適用した後，トークンをスペースで連結してファイルに保存せよ．

In [1]:
def tokenize(text):
    return [token.strip('.,!?;:()[]\'"') for token in text.split() if token.strip('.,!?;:()[]\'"')]

In [24]:
with open('./work/q80.txt', 'w') as fw:
    with open('./data/enwiki-20150112-400-r100-10576.txt', 'r') as fr:
        for line in fr:
            if line.rstrip():
                token_list = tokenize(line.rstrip())
                fw.write(' '.join(token_list) + '\n')

In [27]:
!head -5 ./work/q80.txt

Anarchism
Anarchism is a political philosophy that advocates stateless societies often defined as self-governed voluntary institutions but that several authors have defined as more specific institutions based on non-hierarchical free associations Anarchism holds the state to be undesirable unnecessary or harmful While anti-statism is central anarchism entails opposing authority or hierarchical organisation in the conduct of human relations including but not limited to the state system
As a subtle and anti-dogmatic philosophy anarchism draws on many currents of thought and strategy Anarchism does not offer a fixed body of doctrine from a single particular world view instead fluxing and flowing as a philosophy There are many types and traditions of anarchism not all of which are mutually exclusive Anarchist schools of thought can differ fundamentally supporting anything from extreme individualism to complete collectivism Strains of anarchism have often been divided into the categories of

## 81. 複合語からなる国名への対処

英語では，複数の語の連接が意味を成すことがある．例えば，アメリカ合衆国は"United States"，イギリスは"United Kingdom"と表現されるが，"United"や"States"，"Kingdom"という単語だけでは，指し示している概念・実体が曖昧である．そこで，コーパス中に含まれる複合語を認識し，複合語を1語として扱うことで，複合語の意味を推定したい．しかしながら，複合語を正確に認定するのは大変むずかしいので，ここでは複合語からなる国名を認定したい．

インターネット上から国名リストを各自で入手し，80のコーパス中に出現する複合語の国名に関して，スペースをアンダーバーに置換せよ．例えば，"United States"は"United_States"，"Isle of Man"は"Isle_of_Man"になるはずである．

In [6]:
import pandas as pd

countries = pd.read_csv('https://raw.githubusercontent.com/datasets/country-list/master/data.csv')
countries.head()

,Name,Code
0,Afghanistan,AF
1,Åland Islands,AX
2,Albania,AL
3,Algeria,DZ
4,American Samoa,AS


In [9]:
countries['n_tokens'] = countries.Name.apply(tokenize).apply(len)
countries.head()

,Name,Code,n_tokens
0,Afghanistan,AF,1
1,Åland Islands,AX,2
2,Albania,AL,1
3,Algeria,DZ,1
4,American Samoa,AS,2


### pandas.DataFrame.query
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.query.html
### pandas.DataFrame.sort_values
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sort_values.html

In [10]:
# Nameの中身もtokenizeする必要ある？？
countries.query('n_tokens > 1', inplace=True)   # 元のDeraFrameを書き換える
countries.sort_values('n_tokens', ascending=False, inplace=True)   # 降順
countries.head()

,Name,Code,n_tokens
185,"Saint Helena, Ascension and Tristan da Cunha",SH,7
206,South Georgia and the South Sandwich Islands,GS,7
51,"Congo, the Democratic Republic of the",CD,6
131,"Macedonia, the Former Yugoslav Republic of",MK,6
97,Holy See (Vatican City State),VA,5


In [49]:
multiword_countries = [(name, name.replace(' ', '_')) for name in countries.Name]

multi_list = []

def replace_multiword_countries(text, multiword_countries):
    for old, new in multiword_countries:
        if old in text:
            text = text.replace(old, new)
            multi_list.append(new)   # 確認用
    return text

In [50]:
with open('./work/q81.txt', 'w') as fw:
    with open('./work/q80.txt', 'r') as fr:
        for line in fr:
            fw.write(replace_multiword_countries(line.rstrip(), multiword_countries) + '\n')

In [51]:
!head -5 ./work/q81.txt

Anarchism
Anarchism is a political philosophy that advocates stateless societies often defined as self-governed voluntary institutions but that several authors have defined as more specific institutions based on non-hierarchical free associations Anarchism holds the state to be undesirable unnecessary or harmful While anti-statism is central anarchism entails opposing authority or hierarchical organisation in the conduct of human relations including but not limited to the state system
As a subtle and anti-dogmatic philosophy anarchism draws on many currents of thought and strategy Anarchism does not offer a fixed body of doctrine from a single particular world view instead fluxing and flowing as a philosophy There are many types and traditions of anarchism not all of which are mutually exclusive Anarchist schools of thought can differ fundamentally supporting anything from extreme individualism to complete collectivism Strains of anarchism have often been divided into the categories of

In [52]:
print(multi_list[:20])

['United_States', 'United_States', 'United_States', 'United_States', 'United_Kingdom', 'United_States', 'United_States', 'United_States', 'United_States', 'United_States', 'United_States', 'United_States', 'Hong_Kong', 'United_States', 'United_States', 'United_States', 'Equatorial_Guinea', 'United_States', 'United_States', 'United_States']


## 82. 文脈の抽出

81で作成したコーパス中に出現するすべての単語$t$に関して，単語$t$と文脈語$c$のペアをタブ区切り形式ですべて書き出せ．ただし，文脈語の定義は次の通りとする．

+ ある単語$t$の前後$d$単語を文脈語$c$として抽出する（ただし，文脈語に単語$t$そのものは含まない）
+ 単語$t$を選ぶ度に，文脈幅$d$は$\{1, 2, 3, 4, 5\}$の範囲でランダムに決める．

In [61]:
def get_context_tokens(tokens, target_i, window_size):
    return tokens[max(0, target_i - window_size) : target_i] + tokens[target_i + 1 : min(len(tokens), target_i + window_size + 1)]

In [65]:
import numpy as np

def generate_word_context_pairs(tokens):
    window_sizes = np.random.randint(1, 6, len(tokens))
    
    for target_i, (token, window_size) in enumerate(zip(tokens, window_sizes)):
        for contecxt in get_context_tokens(tokens, target_i, window_size):
            yield token, contecxt

In [68]:
test_tokens = ['This', 'is', 'a', 'test', 'sentence']

In [77]:
results = [result for result in generate_word_context_pairs(test_tokens)]

In [78]:
results

[('This', 'is'),
 ('is', 'This'),
 ('is', 'a'),
 ('a', 'is'),
 ('a', 'test'),
 ('test', 'is'),
 ('test', 'a'),
 ('test', 'sentence'),
 ('sentence', 'This'),
 ('sentence', 'is'),
 ('sentence', 'a'),
 ('sentence', 'test')]

## 83. 単語／文脈の頻度の計測

82の出力を利用し，以下の出現分布，および定数を求めよ．

+ $f(t,c)$: 単語$t$と文脈語$c$の共起回数
+ $f(t,*)$: 単語$t$の出現回数
+ $f(*,c)$: 文脈語$c$の出現回数
+ $N$: 単語と文脈語のペアの総出現回数

In [ ]:
from collections import Counter:
    

## 84. 単語文脈行列の作成

83の出力を利用し，単語文脈行列$X$を作成せよ．ただし，行列$X$の各要素$X_{tc}$は次のように定義する．

+ $f(t, c) \geq 10$ならば，$X_{tc} = {\rm PPMI}(t, c) = \max\{\log \frac{N \times f(t,c)}{f(t,*) \times f(*,c)}, 0\}$
+ $f(t, c) < 10$ならば，$X_{tc} = 0$

ここで，${\rm PPMI}(t, c)$はPositive Pointwise Mutual Information（正の相互情報量）と呼ばれる統計量である．なお，行列$X$の行数・列数は数百万オーダとなり，行列のすべての要素を主記憶上に載せることは無理なので注意すること．幸い，行列$X$のほとんどの要素は$0$になるので，非$0$の要素だけを書き出せばよい．

## 85. 主成分分析による次元圧縮

84で得られた単語文脈行列に対して，主成分分析を適用し，単語の意味ベクトルを300次元に圧縮せよ．


## 86. 単語ベクトルの表示

85で得た単語の意味ベクトルを読み込み，"United States"のベクトルを表示せよ．ただし，"United States"は内部的には"United_States"と表現されていることに注意せよ．


## 87. 単語の類似度

85で得た単語の意味ベクトルを読み込み，"United States"と"U.S."のコサイン類似度を計算せよ．ただし，"U.S."は内部的に"U.S"と表現されていることに注意せよ．

## 88. 類似度の高い単語10件

85で得た単語の意味ベクトルを読み込み，"England"とコサイン類似度が高い10語と，その類似度を出力せよ．

## 89. 加法構成性によるアナロジー

85で得た単語の意味ベクトルを読み込み，vec("Spain") - vec("Madrid") + vec("Athens")を計算し，そのベクトルと類似度の高い10語とその類似度を出力せよ．
